# Bayesian Forecaster

#### Imports

In [1]:
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import pandas as pd
import arviz as az

#### data setup

In [2]:
data_directory = '../data/experiment_small/'
data = np.load(data_directory + 'network_params/data_network.npy')
data = np.transpose(data, (2,1, 0))

data = data.reshape(len(data),2*len(data[0]))
data_future = data[2556:,:]
data = data[:2556,:]

T,D = data.shape
h = 7
f = 7

#### training the model

In [18]:
X = []
y = []
    
for t in range(h, T):
    X.append(data[t-h:t].flatten())
    y.append(data[t])
    
X = np.array(X)
y = np.array(y)

with pm.Model() as model:
    betas = pm.Normal("betas", mu=0, sigma=10, shape=(h*D,D))
    intercepts = pm.Normal("intercepts", mu=0, sigma=10, shape=(D,))
    sigma = pm.HalfNormal("sigma", sigma=1)
    
    mu = pt.dot(X, betas) + intercepts
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

    trace = pm.sample(return_inferencedata=True, target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, intercepts, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3408 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 2 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.


In [19]:
betas_post = trace.posterior["betas"].mean(dim=["chain", "draw"])
intercepts_post = trace.posterior["intercepts"].mean(dim=["chain", "draw"])

X_new = np.zeros((f, h*D))
last_h_steps = data[-h:]
predictions = np.zeros((f, D))

for t in range(f):
    X_new[t] = last_h_steps.flatten()
    predictions[t] = np.dot(X_new[t], betas_post) + intercepts_post
    last_h_steps = np.vstack([last_h_steps[1:], predictions[t]])

predictions = predictions.reshape((f,D//2,2))
predictions

array([[[286.20057111, 226.32225876],
        [334.91758662, 210.7652796 ],
        [ 68.12095809,  63.93669847],
        [ 34.4765381 ,  31.72607054]],

       [[286.69889465, 226.57669101],
        [335.41402917, 211.11032646],
        [ 68.23268781,  64.00719772],
        [ 34.58090782,  31.79709828]],

       [[286.72460265, 226.72434295],
        [335.50979387, 211.31489161],
        [ 68.25793629,  64.07371687],
        [ 34.53864216,  31.78728682]],

       [[286.99548097, 226.75784375],
        [335.87099821, 211.28871138],
        [ 68.35363772,  64.07369052],
        [ 34.61381206,  31.87163995]],

       [[287.41030477, 226.94954171],
        [336.34865321, 211.50002141],
        [ 68.47089567,  64.16338788],
        [ 34.63203916,  31.79929243]],

       [[287.42623198, 227.05323252],
        [336.49369979, 211.65280278],
        [ 68.52347884,  64.21415232],
        [ 34.65776094,  31.89712059]],

       [[287.70406754, 227.01792245],
        [336.67841194, 211.4478832 ],


#### results

In [20]:
data_future = data_future[:7,:].reshape((f,D//2,2))
data_future

array([[[285.08782445, 226.35593399],
        [333.51207395, 210.82476448],
        [ 67.81355933,  63.9587202 ],
        [ 34.34047957,  31.73911717]],

       [[282.02632141, 223.33556313],
        [329.78180072, 207.14461486],
        [ 66.98374545,  63.14005405],
        [ 33.91434925,  31.3187122 ]],

       [[280.939363  , 227.91045409],
        [328.45749724, 212.71880231],
        [ 66.68913401,  64.38005446],
        [ 33.76305164,  31.95549417]],

       [[284.05250617, 221.50776946],
        [332.2506046 , 204.91760932],
        [ 67.53294283,  62.64464874],
        [ 34.19637738,  31.06429449]],

       [[283.82008646, 215.31442634],
        [331.96739713, 197.37144083],
        [ 67.46993389,  60.96596318],
        [ 34.16402179,  30.20223722]],

       [[285.51145892, 212.98253932],
        [334.02823702, 194.53021157],
        [ 67.92838777,  60.33392986],
        [ 34.39945366,  29.87765492]],

       [[278.36305142, 213.14718582],
        [325.31838415, 194.73082155],


In [ ]:
trace.to_netcdf("bayesian_model.nc")